In [1]:
#Librerias:
!pip install spacy
!pip install pandas
!python -m spacy download es_core_news_sm
!python -m spacy download es_core_news_lg

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 6.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 7.1 MB/s eta 0:00:0000:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')


In [17]:
#hacer tanto para el dataset de entrenamiento como para el de testeo. No olvidar cambiar los nombres para que no se pisen
import spacy
from spacy.tokens import DocBin
import json

nlp = spacy.blank("es")

# Lee el archivo JSON
with open('./train_data.json', 'r', encoding='utf-8') as file: #cambiar
    data = json.load(file)

# Procesa los datos para obtener el formato deseado
training_data = []
for example in data:
    for paragraph in example['paragraphs']:
        for sentence in paragraph['sentences']:
            text = sentence['text']
            entities = [(entity['start'], entity['end'], entity['label']) for entity in sentence['entities']]
            training_data.append((text, entities))
# the DocBin will store the example documents
#print(training_data)
db = DocBin()
for text, annotations in training_data:
    doc = nlp(text)
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        ents.append(span)
    doc.ents =  [ent for ent in ents if ent is not None]
    db.add(doc)
db.to_disk("./train.spacy")#cambiar

In [3]:
#Inicializar archivo .conf
!python -m spacy init config config.cfg --lang es --pipeline ner --optimize accuracy


✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.



In [19]:
#Iniciar Entrenamiento
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./test.spacy #dev son data de testeo

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     43.00    0.00    0.00    0.00    0.00
200     200          5.39    607.44   57.14   66.67   50.00    0.57
400     400          0.00      0.00   57.14   66.67   50.00    0.57
600     600          0.00      0.00   57.14   66.67   50.00    0.57
800     800          0.00      0.00   57.14   66.67   50.00    0.57
1000    1000          0.00      0.00   57.14   66.67   50.00    0.57
1200    1200          0.00      0.00   57.14   66.67   50.00    0.57
1400    1400          0.00      0.00   57.14   66.67   50.00    0.57
1600    1600          0.00  

In [22]:
#Test del modelo
import spacy

# Cargar el modelo entrenado
nlp = spacy.load("./output/model-best")

# Probar el modelo
doc = nlp("La Torre Eiffel mide 324 metros de alto")
for ent in doc.ents:
    print(ent.text, ent.label_)
print("FIN")

Torre Eiffel BUILDING
FIN
